# Identify idiom for a specific phrase - Regex and languages models

## Loading the data

In [1]:
import pandas as pd

In [2]:
portuguese_data = pd.read_csv("dataset/stackoverflow_portugues.csv")
portuguese_data.head(6)

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191
5,3864,Como prevenir injeção de código SQL no meu cód...,<p>Desenvolvi uma página em PHP para uso inter...,<php><mysql><sql><sql-injection>,180,10243


In [3]:
portuguese_questions = portuguese_data.Questão[5]
print(portuguese_questions)

<p>Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:</p>

<pre><code>//----CONSULTA SQL----//
$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');
</code></pre>

<p>Logo, digamos que o usuário usar a sentença: <code>1); DROP TABLE Produtos;</code> para ao campo <code>valor</code> o comando ficaria: </p>

<pre><code>insert into Produtos (coluna) values(1); DROP TABLE Produtos;
</code></pre>

<p>Ele vai inserir um novo registro cujo o campo <code>coluna</code> será <code>1</code> e logo em seguida ele vai deletar a tabela Produtos.</p>

<p>Como posso melhorar meu código para prevenir essa situação?</p>



We can notice that we can take of the punctuation, the HTML elements and the codes

## Regex: introduction

A regular expression is a sequence of characters that specifies a search pattern in text. Usually such patterns are used by string-searching algorithms for "find" or "find and replace" operations on strings, or for input validation.

Read more this two sites: [Python RegEx Meta Characters](https://www.w3schools.com/python/gloss_python_regex_metacharacters.asp) and [Regular Expressions](https://www3.ntu.edu.sg/home/ehchua/programming/howto/Regexe.html#:~:text=In%20regex%2C%20the%20uppercase%20metacharacter,%5E0%2D9%5D%20).

### Loading english and spanish data

In [4]:
english_data = pd.read_csv("dataset/stackoverflow_ingles.csv")
english_data.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,11227809,Why is it faster to process a sorted array tha...,<p>Here is a piece of C++ code that seems very...,<java><c++><performance><optimization><branch-...,23057,1358574
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,<git><version-control><git-commit><undo>,19640,7906137
2,2003505,How do I delete a Git branch locally and remot...,<p>I want to delete a branch both locally and ...,<git><git-branch><git-remote>,15249,6940906
3,292357,What is the difference between 'git pull' and ...,<blockquote>\n <p><strong>Moderator Note:</st...,<git><git-pull><git-fetch>,11008,2543052
4,477816,What is the correct JSON content type?,"<p>I've been messing around with <a href=""http...",<json><http-headers><content-type>,9701,2478940


In [5]:
english_question = english_data.Questão[0]
print(english_question)

<p>Here is a piece of C++ code that seems very peculiar. For some strange reason, sorting the data miraculously makes the code almost six times faster.</p>

<pre class="lang-cpp prettyprint-override"><code>#include &lt;algorithm&gt;
#include &lt;ctime&gt;
#include &lt;iostream&gt;

int main()
{
    // Generate data
    const unsigned arraySize = 32768;
    int data[arraySize];

    for (unsigned c = 0; c &lt; arraySize; ++c)
        data[c] = std::rand() % 256;

    // !!! With this, the next loop runs faster
    std::sort(data, data + arraySize);

    // Test
    clock_t start = clock();
    long long sum = 0;

    for (unsigned i = 0; i &lt; 100000; ++i)
    {
        // Primary loop
        for (unsigned c = 0; c &lt; arraySize; ++c)
        {
            if (data[c] &gt;= 128)
                sum += data[c];
        }
    }

    double elapsedTime = static_cast&lt;double&gt;(clock() - start) / CLOCKS_PER_SEC;

    std::cout &lt;&lt; elapsedTime &lt;&lt; std::endl;
    std::cout &lt

In [6]:
spanish_data = pd.read_csv("dataset/stackoverflow_espanhol.csv")
spanish_data.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,18232,¿Cómo evitar la inyección SQL en PHP?,<p>Las sentencias dinámicas son sentencias SQL...,<php><mysql><sql><seguridad><inyección-sql>,169,38614
1,197,¿Por qué mis programas no pueden hacer cálculo...,<p>Unas veces los cálculos funcionan correctam...,<matemáticas><coma-flotante><lenguaje-agnóstico>,141,3860
2,36,¿Cuál es la diferencia entre un inner y un out...,<p>¿Cuál es la diferencia entre un <code>inner...,<mysql><sql><join>,97,53627
3,29177,¿Por qué es considerado una mala práctica util...,"<p>La mayoría de nosotros decimos, (muchas vec...",<variables><variables-globales><patrones-de-di...,89,9987
4,142,Validar un email en JavaScript que acepte todo...,<h3>Pregunta</h3>\n\n<p>¿Cómo validar un e-mai...,<javascript><validación><email><regex>,87,73129


In [7]:
spanish_question = spanish_data.Questão[0]
print(spanish_question)

<p>Las sentencias dinámicas son sentencias SQL que se crean como cadenas de texto (strings) y en las que se insertan/concatenan valores obtenidos de alguna fuente (normalmente proveniente del usuario), lo que puede hacer que sean vulnerables a inyección SQL si no se sanean las entradas, como por ejemplo:</p>

<pre><code>$id_usuario = $_POST["id"];

mysql_query("SELECT * FROM usuarios WHERE id = $id_usuario");
</code></pre>

<p>Eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación (web o no) porque si el usuario introdujese un valor como <code>1; DROP TABLE usuarios;--</code> nos encontraríamos con que la sentencia ejecutada sería:</p>

<pre><code>SELECT * FROM usuarios WHERE id = 1; DROP TABLE usuarios;--
</code></pre>

<p>Y se eliminaría la tabla Usuarios con todos los datos contenidos en ella. </p>

<p><strong>¿Cómo puedo evitar que la inyección SQL ocurra en PHP?</strong></p>



English and Spanish data are in the same format as the Portuguese data

## Using Metacharacters

Metacharacters are considered as the building blocks of regular expressions. Regular expressions are patterns used to match character combinations in the strings. Metacharacter has special meaning in finding patterns and are mostly used to define the search criteria and any text manipulations.

Read [more](https://www.w3schools.com/python/gloss_python_regex_metacharacters.asp).

Test you regex in [regex101](https://regex101.com/).

Library [re](https://docs.python.org/3/library/re.html).

[Regular Expression HOWTO](https://docs.python.org/3/howto/regex.html#regex-howto).

In [8]:
import re
re.findall(r"<p>", portuguese_questions)

['<p>', '<p>', '<p>', '<p>']

In [9]:
# Find all HTML tags
re.findall(r"<.*?>", portuguese_questions) # <.*> stops just for a newline character

['<p>',
 '</p>',
 '<pre>',
 '<code>',
 '</code>',
 '</pre>',
 '<p>',
 '<code>',
 '</code>',
 '<code>',
 '</code>',
 '</p>',
 '<pre>',
 '<code>',
 '</code>',
 '</pre>',
 '<p>',
 '<code>',
 '</code>',
 '<code>',
 '</code>',
 '</p>',
 '<p>',
 '</p>']

In [23]:
# Find all regex and substitute for something
test_text = re.sub(r"<.*?>","  T----E----S----T  ", portuguese_questions)
print(test_text)

  T----E----S----T  Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:  T----E----S----T  

  T----E----S----T    T----E----S----T  //----CONSULTA SQL----//
$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');
  T----E----S----T    T----E----S----T  

  T----E----S----T  Logo, digamos que o usuário usar a sentença:   T----E----S----T  1); DROP TABLE Produtos;  T----E----S----T   para ao campo   T----E----S----T  valor  T----E----S----T   o comando ficaria:   T----E----S----T  

  T----E----S----T    T----E----S----T  insert into Produtos (coluna) values(1); DROP TABLE Produtos;
  T----E----S----T    T----E----S----T  

  T----E----S----T  Ele vai inserir um novo

In [24]:
# Search the pattern and returns the position (return an object re.Match - see more in the documentation)
re.search(r"70", "18728736187263817628631872638716283670")

<re.Match object; span=(36, 38), match='70'>

In [25]:
# Transform a string in a regex
regex = re.compile(r"70")

In [26]:
regex.search("18728736187263817628631872638716283670")

<re.Match object; span=(36, 38), match='70'>

Analyzing the performance of using re.compile and regex.search rather than re.search just once

In [10]:
from timeit import timeit

In [28]:
setup = """import re"""
timeit("""re.search(r"70","18728736187263817628631872638716283670")""",
      setup)

0.4425977470000362

In [29]:
setup = """import re
regex = re.compile(r"70")"""

timeit("""regex.search("18728736187263817628631872638716283670")""",
      setup)

0.08850591700138466

Using the function re.compile as best pratice in the project

## Preprocessing web data

### Removing the tags HTML

In [11]:
# Function for just one string of a list of strings
def remove_func(texts, regex):
    if type(texts) == str:
        return regex.sub("", texts) 
    else:
        return [regex.sub("", text) for text in texts]

In [12]:
regex_html = re.compile(r"<.*?>")
question_without_tag = remove_func(english_question, regex_html)
print(question_without_tag)

Here is a piece of C++ code that seems very peculiar. For some strange reason, sorting the data miraculously makes the code almost six times faster.

#include &lt;algorithm&gt;
#include &lt;ctime&gt;
#include &lt;iostream&gt;

int main()
{
    // Generate data
    const unsigned arraySize = 32768;
    int data[arraySize];

    for (unsigned c = 0; c &lt; arraySize; ++c)
        data[c] = std::rand() % 256;

    // !!! With this, the next loop runs faster
    std::sort(data, data + arraySize);

    // Test
    clock_t start = clock();
    long long sum = 0;

    for (unsigned i = 0; i &lt; 100000; ++i)
    {
        // Primary loop
        for (unsigned c = 0; c &lt; arraySize; ++c)
        {
            if (data[c] &gt;= 128)
                sum += data[c];
        }
    }

    double elapsedTime = static_cast&lt;double&gt;(clock() - start) / CLOCKS_PER_SEC;

    std::cout &lt;&lt; elapsedTime &lt;&lt; std::endl;
    std::cout &lt;&lt; "sum = " &lt;&lt; sum &lt;&lt; std::endl;
}



Wit

### Taking off the code using the HTML code 

In [36]:
print(english_question)

<p>Here is a piece of C++ code that seems very peculiar. For some strange reason, sorting the data miraculously makes the code almost six times faster.</p>

<pre class="lang-cpp prettyprint-override"><code>#include &lt;algorithm&gt;
#include &lt;ctime&gt;
#include &lt;iostream&gt;

int main()
{
    // Generate data
    const unsigned arraySize = 32768;
    int data[arraySize];

    for (unsigned c = 0; c &lt; arraySize; ++c)
        data[c] = std::rand() % 256;

    // !!! With this, the next loop runs faster
    std::sort(data, data + arraySize);

    // Test
    clock_t start = clock();
    long long sum = 0;

    for (unsigned i = 0; i &lt; 100000; ++i)
    {
        // Primary loop
        for (unsigned c = 0; c &lt; arraySize; ++c)
        {
            if (data[c] &gt;= 128)
                sum += data[c];
        }
    }

    double elapsedTime = static_cast&lt;double&gt;(clock() - start) / CLOCKS_PER_SEC;

    std::cout &lt;&lt; elapsedTime &lt;&lt; std::endl;
    std::cout &lt

In [13]:
def substitute_code(texts, regex):
    if type(texts) == str:
        return regex.sub("CODE", texts)
    else:
        return [regex.sub("CODE", text) for text in texts]

In [14]:
english_question

'<p>Here is a piece of C++ code that seems very peculiar. For some strange reason, sorting the data miraculously makes the code almost six times faster.</p>\n\n<pre class="lang-cpp prettyprint-override"><code>#include &lt;algorithm&gt;\n#include &lt;ctime&gt;\n#include &lt;iostream&gt;\n\nint main()\n{\n    // Generate data\n    const unsigned arraySize = 32768;\n    int data[arraySize];\n\n    for (unsigned c = 0; c &lt; arraySize; ++c)\n        data[c] = std::rand() % 256;\n\n    // !!! With this, the next loop runs faster\n    std::sort(data, data + arraySize);\n\n    // Test\n    clock_t start = clock();\n    long long sum = 0;\n\n    for (unsigned i = 0; i &lt; 100000; ++i)\n    {\n        // Primary loop\n        for (unsigned c = 0; c &lt; arraySize; ++c)\n        {\n            if (data[c] &gt;= 128)\n                sum += data[c];\n        }\n    }\n\n    double elapsedTime = static_cast&lt;double&gt;(clock() - start) / CLOCKS_PER_SEC;\n\n    std::cout &lt;&lt; elapsedTime &l

In [15]:
regex_code = re.compile(r"<code>(.|(\n))*?</code>")

In [16]:
question_without_code = substitute_code(spanish_question,regex_code)
print(question_without_code)

<p>Las sentencias dinámicas son sentencias SQL que se crean como cadenas de texto (strings) y en las que se insertan/concatenan valores obtenidos de alguna fuente (normalmente proveniente del usuario), lo que puede hacer que sean vulnerables a inyección SQL si no se sanean las entradas, como por ejemplo:</p>

<pre>CODE</pre>

<p>Eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación (web o no) porque si el usuario introdujese un valor como CODE nos encontraríamos con que la sentencia ejecutada sería:</p>

<pre>CODE</pre>

<p>Y se eliminaría la tabla Usuarios con todos los datos contenidos en ella. </p>

<p><strong>¿Cómo puedo evitar que la inyección SQL ocurra en PHP?</strong></p>



In [17]:
portuguese_data.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191


In [18]:
questions_port_without_code = substitute_code(
    portuguese_data.Questão,
    regex_code
)

questions_port_without_code_tag = remove_func(
    questions_port_without_code,
    regex_html
)

portuguese_data["without_code_tag"] = questions_port_without_code_tag

In [19]:
portuguese_data.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,without_code_tag
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367,Se eu fizer o hash de senhas antes de armazená...
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953,Qual é a diferença entre CODE e CODE? Podem me...
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761,Uma dúvida muito comum é por que devemos parar...
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075,É comum encontrar uma mensagem de erro que diz...
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191,Me parecem termos muito próximos e eventualmen...


In [20]:
questions_ing_without_code = substitute_code(
    english_data.Questão,
    regex_code
)

questions_ing_without_code_tag = remove_func(
    questions_ing_without_code,
    regex_html
)

english_data["without_code_tag"] = questions_ing_without_code_tag
english_data.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,without_code_tag
0,11227809,Why is it faster to process a sorted array tha...,<p>Here is a piece of C++ code that seems very...,<java><c++><performance><optimization><branch-...,23057,1358574,Here is a piece of C++ code that seems very pe...
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,<git><version-control><git-commit><undo>,19640,7906137,I accidentally committed the wrong files to Gi...
2,2003505,How do I delete a Git branch locally and remot...,<p>I want to delete a branch both locally and ...,<git><git-branch><git-remote>,15249,6940906,I want to delete a branch both locally and rem...
3,292357,What is the difference between 'git pull' and ...,<blockquote>\n <p><strong>Moderator Note:</st...,<git><git-pull><git-fetch>,11008,2543052,\n Moderator Note: Given that this question h...
4,477816,What is the correct JSON content type?,"<p>I've been messing around with <a href=""http...",<json><http-headers><content-type>,9701,2478940,I've been messing around with JSON for some ti...


In [21]:
questions_span_without_code = substitute_code(
    spanish_data.Questão,
    regex_code
)

questions_span_without_code_tag = remove_func(
    questions_span_without_code,
    regex_html
)

spanish_data["without_code_tag"] = questions_span_without_code_tag
spanish_data.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,without_code_tag
0,18232,¿Cómo evitar la inyección SQL en PHP?,<p>Las sentencias dinámicas son sentencias SQL...,<php><mysql><sql><seguridad><inyección-sql>,169,38614,Las sentencias dinámicas son sentencias SQL qu...
1,197,¿Por qué mis programas no pueden hacer cálculo...,<p>Unas veces los cálculos funcionan correctam...,<matemáticas><coma-flotante><lenguaje-agnóstico>,141,3860,Unas veces los cálculos funcionan correctament...
2,36,¿Cuál es la diferencia entre un inner y un out...,<p>¿Cuál es la diferencia entre un <code>inner...,<mysql><sql><join>,97,53627,¿Cuál es la diferencia entre un CODE y un CODE...
3,29177,¿Por qué es considerado una mala práctica util...,"<p>La mayoría de nosotros decimos, (muchas vec...",<variables><variables-globales><patrones-de-di...,89,9987,"La mayoría de nosotros decimos, (muchas veces ..."
4,142,Validar un email en JavaScript que acepte todo...,<h3>Pregunta</h3>\n\n<p>¿Cómo validar un e-mai...,<javascript><validación><email><regex>,87,73129,Pregunta\n\n¿Cómo validar un e-mail que acepte...


In [22]:
spanish_data.without_code_tag[0]

'Las sentencias dinámicas son sentencias SQL que se crean como cadenas de texto (strings) y en las que se insertan/concatenan valores obtenidos de alguna fuente (normalmente proveniente del usuario), lo que puede hacer que sean vulnerables a inyección SQL si no se sanean las entradas, como por ejemplo:\n\nCODE\n\nEso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación (web o no) porque si el usuario introdujese un valor como CODE nos encontraríamos con que la sentencia ejecutada sería:\n\nCODE\n\nY se eliminaría la tabla Usuarios con todos los datos contenidos en ella. \n\n¿Cómo puedo evitar que la inyección SQL ocurra en PHP?\n'

In [23]:
print(spanish_data.without_code_tag[0])

Las sentencias dinámicas son sentencias SQL que se crean como cadenas de texto (strings) y en las que se insertan/concatenan valores obtenidos de alguna fuente (normalmente proveniente del usuario), lo que puede hacer que sean vulnerables a inyección SQL si no se sanean las entradas, como por ejemplo:

CODE

Eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación (web o no) porque si el usuario introdujese un valor como CODE nos encontraríamos con que la sentencia ejecutada sería:

CODE

Y se eliminaría la tabla Usuarios con todos los datos contenidos en ella. 

¿Cómo puedo evitar que la inyección SQL ocurra en PHP?



## Removing the digits and the punctuation

In [24]:
regex_punctutation = re.compile(r"[^\w]")
print(remove_func(questions_span_without_code_tag[0], regex_punctutation))

LassentenciasdinámicassonsentenciasSQLquesecreancomocadenasdetextostringsyenlasqueseinsertanconcatenanvaloresobtenidosdealgunafuentenormalmenteprovenientedelusuarioloquepuedehacerqueseanvulnerablesainyecciónSQLsinosesaneanlasentradascomoporejemploCODEEsoesunejemplodeunavulnerabilidadgraveenlaseguridaddeunaaplicaciónwebonoporquesielusuariointrodujeseunvalorcomoCODEnosencontraríamosconquelasentenciaejecutadaseríaCODEYseeliminaríalatablaUsuarioscontodoslosdatoscontenidosenellaCómopuedoevitarquelainyecciónSQLocurraenPHP


In [25]:
regex_punctutation = re.compile(r"[^\w\s]")
print(remove_func(questions_span_without_code_tag[0], regex_punctutation))

Las sentencias dinámicas son sentencias SQL que se crean como cadenas de texto strings y en las que se insertanconcatenan valores obtenidos de alguna fuente normalmente proveniente del usuario lo que puede hacer que sean vulnerables a inyección SQL si no se sanean las entradas como por ejemplo

CODE

Eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación web o no porque si el usuario introdujese un valor como CODE nos encontraríamos con que la sentencia ejecutada sería

CODE

Y se eliminaría la tabla Usuarios con todos los datos contenidos en ella 

Cómo puedo evitar que la inyección SQL ocurra en PHP



### Normalizing the text

In [26]:
def lower_case(texts):
    if type(texts) == str:
        return texts.lower()
    else:
        return [text.lower() for text in texts]
print(lower_case(questions_span_without_code_tag[0]))

las sentencias dinámicas son sentencias sql que se crean como cadenas de texto (strings) y en las que se insertan/concatenan valores obtenidos de alguna fuente (normalmente proveniente del usuario), lo que puede hacer que sean vulnerables a inyección sql si no se sanean las entradas, como por ejemplo:

code

eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación (web o no) porque si el usuario introdujese un valor como code nos encontraríamos con que la sentencia ejecutada sería:

code

y se eliminaría la tabla usuarios con todos los datos contenidos en ella. 

¿cómo puedo evitar que la inyección sql ocurra en php?



In [27]:
regex_digits = re.compile(r"\d+")
print(remove_func("Alura \n 1234 Caelum 1234", regex_digits))

Alura 
  Caelum 


### Substitute whitespaces

In [28]:
def substitute_for_whitespace(texts, regex):
    if type(texts) == str:
        return regex.sub(" ", texts)
    else:
        return [regex.sub(" ", text) for text in texts]

In [29]:
regex_space = re.compile(r" +")
print(substitute_for_whitespace("Alura \n \n     Caleum", regex_space))

Alura 
 
 Caleum


In [30]:
regex_newline = re.compile(r"(\n)") 
print(substitute_for_whitespace("Alura \n \n     Caleum", regex_newline))

Alura         Caleum


## Applying all the filters for portuguese, english and spanish texts

In [31]:
def remove_func(texts, regex):
    if type(texts) == str:
        return regex.sub("", texts) 
    else:
        return [regex.sub("", text) for text in texts]

def lower_case(texts):
    if type(texts) == str:
        return texts.lower()
    else:
        return [text.lower() for text in texts]

def substitute_for_whitespace(texts, regex):
    if type(texts) == str:
        return regex.sub(" ", texts)
    else:
        return [regex.sub(" ", text) for text in texts]

In [33]:
# Portuguese texts

# Removing the punctuation
regex_punctutation = re.compile(r"[^\w\s]")
questions_port_without_punct = remove_func(
    portuguese_data.without_code_tag, 
    regex_punctutation
)

# Normalizing the text
questions_port_without_punct_lower = lower_case(questions_port_without_punct)

# Removing the digits
regex_digits = re.compile(r"\d+")
questions_port_without_punct_lower_dig = remove_func(
    questions_port_without_punct_lower,
    regex_digits
)

# Removing the newlines
regex_newline = re.compile(r"(\n)") 
questions_port_without_newline = substitute_for_whitespace(
    questions_port_without_punct_lower_dig,
    regex_newline
)

# Removing unnecessary spaces
regex_space = re.compile(r" +")
questions_port_without_whitespaces_duplicated = substitute_for_whitespace(
    questions_port_without_newline,
    regex_space
)

portuguese_data["preprocessed_questions"] = questions_port_without_whitespaces_duplicated

In [34]:
portuguese_data.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,without_code_tag,preprocessed_questions
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367,Se eu fizer o hash de senhas antes de armazená...,se eu fizer o hash de senhas antes de armazená...
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953,Qual é a diferença entre CODE e CODE? Podem me...,qual é a diferença entre code e code podem me ...
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761,Uma dúvida muito comum é por que devemos parar...,uma dúvida muito comum é por que devemos parar...
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075,É comum encontrar uma mensagem de erro que diz...,é comum encontrar uma mensagem de erro que diz...
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191,Me parecem termos muito próximos e eventualmen...,me parecem termos muito próximos e eventualmen...


In [35]:
portuguese_data.preprocessed_questions[0]

'se eu fizer o hash de senhas antes de armazenálas em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém estou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque como força bruta na página de login da aplicação keylogger no cliente e criptoanálise rubberhose qualquer forma de hash não vai impedir esses ataques tenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido como dar maior garantia de segurança neste aspecto quais preocupações adicionais evitariam o acesso às senhas existem formas melhores de fazer esse hash '

In [36]:
# English texts

# Removing the punctuation
questions_eng_without_punct = remove_func(
    english_data.without_code_tag, 
    regex_punctutation
)

# Normalizing the text
questions_eng_without_punct_lower = lower_case(questions_eng_without_punct)

# Removing the digits
questions_eng_without_punct_lower_dig = remove_func(
    questions_eng_without_punct_lower,
    regex_digits
)

# Removing the newlines
questions_eng_without_newline = substitute_for_whitespace(
    questions_eng_without_punct_lower_dig,
    regex_newline
)

# Removing unnecessary spaces
questions_eng_without_newline = substitute_for_whitespace(
    questions_eng_without_newline,
    regex_space
)

# Creating a new column
english_data["preprocessed_questions"] = questions_eng_without_newline

In [37]:
english_data.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,without_code_tag,preprocessed_questions
0,11227809,Why is it faster to process a sorted array tha...,<p>Here is a piece of C++ code that seems very...,<java><c++><performance><optimization><branch-...,23057,1358574,Here is a piece of C++ code that seems very pe...,here is a piece of c code that seems very pecu...
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,<git><version-control><git-commit><undo>,19640,7906137,I accidentally committed the wrong files to Gi...,i accidentally committed the wrong files to gi...
2,2003505,How do I delete a Git branch locally and remot...,<p>I want to delete a branch both locally and ...,<git><git-branch><git-remote>,15249,6940906,I want to delete a branch both locally and rem...,i want to delete a branch both locally and rem...
3,292357,What is the difference between 'git pull' and ...,<blockquote>\n <p><strong>Moderator Note:</st...,<git><git-pull><git-fetch>,11008,2543052,\n Moderator Note: Given that this question h...,moderator note given that this question has a...
4,477816,What is the correct JSON content type?,"<p>I've been messing around with <a href=""http...",<json><http-headers><content-type>,9701,2478940,I've been messing around with JSON for some ti...,ive been messing around with json for some tim...


In [38]:
print(english_data.preprocessed_questions[0])

here is a piece of c code that seems very peculiar for some strange reason sorting the data miraculously makes the code almost six times faster code without code the code runs in seconds with the sorted data the code runs in seconds initially i thought this might be just a language or compiler anomaly so i tried it in java code with a somewhat similar but less extreme result my first thought was that sorting brings the data into the cache but then i thought how silly that is because the array was just generated what is going on why is it faster to process a sorted array than an unsorted array the code is summing up some independent terms and the order should not matter 


In [39]:
# Spanish texts

# Removing the punctuation
questions_span_without_punct = remove_func(
    spanish_data.without_code_tag, 
    regex_punctutation)

# Normalizing the text
questions_span_without_punct_lower = lower_case(questions_span_without_punct)

# Removing the digits
questions_span_without_punct_lower_dig = remove_func(
    questions_span_without_punct_lower,
    regex_digits
)

# Removing the newlines
questions_span_without_newline = substitute_for_whitespace(
    questions_span_without_punct_lower_dig,
    regex_newline
)

# Removing unnecessary spaces
questions_span_without_newline = substitute_for_whitespace(
    questions_span_without_newline,
    regex_space
)

# Creating a new column
spanish_data["preprocessed_questions"] = questions_span_without_newline

In [40]:
spanish_data.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,without_code_tag,preprocessed_questions
0,18232,¿Cómo evitar la inyección SQL en PHP?,<p>Las sentencias dinámicas son sentencias SQL...,<php><mysql><sql><seguridad><inyección-sql>,169,38614,Las sentencias dinámicas son sentencias SQL qu...,las sentencias dinámicas son sentencias sql qu...
1,197,¿Por qué mis programas no pueden hacer cálculo...,<p>Unas veces los cálculos funcionan correctam...,<matemáticas><coma-flotante><lenguaje-agnóstico>,141,3860,Unas veces los cálculos funcionan correctament...,unas veces los cálculos funcionan correctament...
2,36,¿Cuál es la diferencia entre un inner y un out...,<p>¿Cuál es la diferencia entre un <code>inner...,<mysql><sql><join>,97,53627,¿Cuál es la diferencia entre un CODE y un CODE...,cuál es la diferencia entre un code y un code ...
3,29177,¿Por qué es considerado una mala práctica util...,"<p>La mayoría de nosotros decimos, (muchas vec...",<variables><variables-globales><patrones-de-di...,89,9987,"La mayoría de nosotros decimos, (muchas veces ...",la mayoría de nosotros decimos muchas veces si...
4,142,Validar un email en JavaScript que acepte todo...,<h3>Pregunta</h3>\n\n<p>¿Cómo validar un e-mai...,<javascript><validación><email><regex>,87,73129,Pregunta\n\n¿Cómo validar un e-mail que acepte...,pregunta cómo validar un email que acepte todo...


In [41]:
print(spanish_data.preprocessed_questions[0])

las sentencias dinámicas son sentencias sql que se crean como cadenas de texto strings y en las que se insertanconcatenan valores obtenidos de alguna fuente normalmente proveniente del usuario lo que puede hacer que sean vulnerables a inyección sql si no se sanean las entradas como por ejemplo code eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación web o no porque si el usuario introdujese un valor como code nos encontraríamos con que la sentencia ejecutada sería code y se eliminaría la tabla usuarios con todos los datos contenidos en ella cómo puedo evitar que la inyección sql ocurra en php 


In [42]:
phrase_test = "Hey! Are you ok?"
regex = re.compile(r"[^\w\s]")
regex.sub("", phrase_test)

'Hey Are you ok'

## NLTK functions

A bigram or digram is a sequence of two adjacent elements from a string of tokens, which are typically letters, syllables, or words. A bigram is an n-gram for n=2.

In [44]:
from nltk.util import bigrams

# The first and last "a" appear just once
test_text = "alura"
print(list(bigrams(test_text)))

[('a', 'l'), ('l', 'u'), ('u', 'r'), ('r', 'a')]


In [45]:
from nltk.lm.preprocessing import pad_both_ends

# The first and last "a" appear twice as the other letters
# I can notice how many times a letter begins a phrase (Fakechars)
print(list(bigrams(pad_both_ends(test_text, n=2))))

[('<s>', 'a'), ('a', 'l'), ('l', 'u'), ('u', 'r'), ('r', 'a'), ('a', '</s>')]


## Creating a ML model

In [46]:
portuguese_data["idiom"] = "port"
spanish_data["idiom"] = "span"
english_data["idiom"] = "eng"

portuguese_data.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,without_code_tag,preprocessed_questions,idiom
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367,Se eu fizer o hash de senhas antes de armazená...,se eu fizer o hash de senhas antes de armazená...,port
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953,Qual é a diferença entre CODE e CODE? Podem me...,qual é a diferença entre code e code podem me ...,port
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761,Uma dúvida muito comum é por que devemos parar...,uma dúvida muito comum é por que devemos parar...,port
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075,É comum encontrar uma mensagem de erro que diz...,é comum encontrar uma mensagem de erro que diz...,port
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191,Me parecem termos muito próximos e eventualmen...,me parecem termos muito próximos e eventualmen...,port


In [48]:
len(spanish_data)

500

In [49]:
from sklearn.model_selection import train_test_split

port_train, port_test = train_test_split(
    portuguese_data.preprocessed_questions,
    test_size = 0.2,
    random_state = 123
)

In [50]:
from sklearn.model_selection import train_test_split

span_train, span_test = train_test_split(
    spanish_data.preprocessed_questions,
    test_size = 0.2,
    random_state = 123
)


In [51]:
from sklearn.model_selection import train_test_split

eng_train, eng_test = train_test_split(
    english_data.preprocessed_questions,
    test_size = 0.2,
    random_state = 123
)

In [52]:
all_questions_port = ' '.join(port_train)

In [55]:
from nltk.tokenize import WhitespaceTokenizer

# Separate all the words by whitespace
all_words_port = WhitespaceTokenizer().tokenize(all_questions_port)

In [56]:
from nltk.lm.preprocessing import padded_everygram_pipeline

# Put fakechart in each word and build a bigram
port_train_bigram, vocab_port = padded_everygram_pipeline(2, all_words_port)

The first are the n-grams of the sentences passed as a parameter. The second parameter returned by the serious function is the words of the sentences passed as a parameter, for example let's pass several sentences, will the words of these sentences be returned, since if we pass several words (as done in the course), will several letters of these words be returned? Only point, if you just pass the text and not a list of words or fake char, one character will be added each (if it's bgram every 2 characters).

### Creating a MLE model

In statistics, **maximum likelihood estimation (MLE)** is a method of estimating the parameters of an assumed probability distribution, given some observed data. This is achieved by maximizing a likelihood function so that, under the assumed statistical model, the observed data is most probable. The point in the parameter space that maximizes the likelihood function is called the maximum likelihood estimate. The logic of maximum likelihood is both intuitive and flexible, and as such the method has become a dominant means of statistical inference

In [60]:
from nltk.lm import MLE

model_port = MLE(2) # Bigram model
model_port.fit(port_train_bigram, vocab_port)

In [61]:
model_port.generate(num_words=6)

['i', 'c', 'o', 'm', '</s>', 'e']

In [62]:
from nltk.lm import NgramCounter

# See the item that appear more after the letter "i"
model_port.counts[['i']].items()

dict_items([('n', 1267), ('c', 817), ('a', 857), ('r', 384), ('s', 1393), ('o', 620), ('t', 718), ('</s>', 601), ('f', 314), ('l', 399), ('d', 651), ('g', 365), ('e', 173), ('p', 277), ('m', 559), ('v', 311), ('z', 370), ('x', 69), ('ç', 85), ('ê', 17), ('k', 15), ('á', 43), ('õ', 6), ('b', 112), ('é', 1), ('q', 30), ('u', 14), ('j', 5), ('ú', 4), ('ã', 3), ('ó', 1), ('h', 1), ('w', 2), ('i', 3)])

In [73]:
# Testing for portuguese
text = "bom dia"
words = WhitespaceTokenizer().tokenize(text)
words_fakechar = [list(pad_both_ends(word, n=2)) for word in words] # add fakechar for each word
words_bigramns = [list(bigrams(word)) for word in words_fakechar] # bigram for each word
print(words_bigramns)

[[('<s>', 'b'), ('b', 'o'), ('o', 'm'), ('m', '</s>')], [('<s>', 'd'), ('d', 'i'), ('i', 'a'), ('a', '</s>')]]


In [74]:
print(words_bigramns[0])

[('<s>', 'b'), ('b', 'o'), ('o', 'm'), ('m', '</s>')]


In information theory, perplexity is a measurement of how well a probability distribution or probability model predicts a sample. It may be used to compare probability models. A low perplexity indicates the probability distribution is good at predicting the sample

In [75]:
# The closer to zero the better.
print(model_port.perplexity(words_bigramns[0]))
print(model_port.perplexity(words_bigramns[1]))

15.299125474590992
8.118290440853844


## Machine learning models train and test

In [76]:
def train_model_mle(list_texts):
    all_questions = ' '.join(list_texts)
    all_words = WhitespaceTokenizer().tokenize(all_questions)
    bigrams, vocabulary = padded_everygram_pipeline(2, all_words)
    model = MLE(2)
    model.fit(bigrams, vocabulary)
    return model

In [77]:
model_port_2 = train_model_mle(port_train)

In [78]:
print(model_port_2.perplexity(words_bigramns[0]))
print(model_port_2.perplexity(words_bigramns[1]))

15.299125474590992
8.118290440853844


In [79]:
model_eng = train_model_mle(eng_train)
model_span = train_model_mle(span_train)

In [80]:
print(model_eng.perplexity(words_bigramns[0]))
print(model_eng.perplexity(words_bigramns[1]))

13.207626654673446
22.271365866014612


In [81]:
13.2+22.27

35.47

### Function to calculate the perplexity

In [82]:
def calculate_perplexity(model, text):
    perplexity = 0
    words = WhitespaceTokenizer().tokenize(text)
    words_fakechar = [list(pad_both_ends(word, n=2)) for word in words]
    words_bigramns = [list(bigrams(word)) for word in words_fakechar]

    for word in words_bigramns:
        perplexity += model.perplexity(word)

    return perplexity

In [87]:
print(calculate_perplexity(model_eng, "bom dia"))

35.47899252068806


In [84]:
print(calculate_perplexity(model_port, port_test.iloc[0]))

2006.9786364086417


In [85]:
port_test.iloc[0]

'até a época em que os computadores eram puramente mecânicos e eram programados por cartões perfurados eu entendo como funciona depois quando surgiram os primeiros computadores digitais operados com válvulas e relés fica mais complicado de compreender como eram feitas as programações imagino que deveria haver algo semelhante a um compartimento com uma matriz de relés semelhante aos cartões perfurados pré programada que continha um programa fixo e único a ser executado pela máquina quando fosse preciso trocar o programa os programadores apenas rearranjavam as válvulas e a máquina continuava a trabalhar como surgiu essa noção que temos de programação hoje que o programador digita códigos em um editor de texto e compila eu não consigo imaginar como chegou a esse ponto por exemplo o assembly ele é apenas traduzido de textos para binário pelo assembler mas poxa seria preciso ter outra linguagem de programação para se criar o assembler suponha que só existam os computadores controlados por r

In [86]:
print(calculate_perplexity(model_eng, port_test.iloc[0]))

inf


## Laplace model

In [88]:
from nltk.lm import Laplace

# To deal with 1/Probability ~= inf (avoid division for zero)
def train_model_laplace(list_texts):
    all_questions = ' '.join(list_texts)
    all_words = WhitespaceTokenizer().tokenize(all_questions)
    bigrams, vocabulary = padded_everygram_pipeline(2, all_words)
    model = Laplace(2)
    model.fit(bigrams, vocabulary)

    return model

Detecting the language

In [89]:
model_eng_Laplace = train_model_laplace(eng_train)
print(calculate_perplexity(model_eng_Laplace, port_test.iloc[0]))

5876.837588345698


In [90]:
model_port_Laplace = train_model_laplace(port_train)
print(calculate_perplexity(model_port_Laplace, port_test.iloc[0]))

2009.1937946178912


In [91]:
model_span_Laplace = train_model_laplace(span_train)
print(calculate_perplexity(model_span_Laplace, port_test.iloc[0]))

3488.5698949157722


So we could detect that the language is portuguese

## Detecting idioms

In [93]:
def atribui_idiom(list_texts):
    idiom = []
    for text in list_texts:
        portuguese = calculate_perplexity(model_port_Laplace, text)
        english = calculate_perplexity(model_eng_Laplace, text)
        spanish = calculate_perplexity(model_span_Laplace, text)
        if english >= portuguese <= spanish:
            idiom.append("portuguese")
        elif portuguese > english < spanish:
            idiom.append("english")
        else:
            idiom.append("spanish")
    return idiom

In [94]:
results_portuguese = atribui_idiom(port_test)

In [95]:
rate_portuguese = results_portuguese.count("portuguese")/len(results_portuguese)

In [96]:
len(port_test)

100

In [98]:
# Get right all the texts
rate_portuguese

1.0

In [99]:
results_english = atribui_idiom(eng_test)
rate_english = results_english.count("english")/len(results_english)
results_spanish = atribui_idiom(span_test)
rate_spanish = results_spanish.count("spanish")/len(results_spanish)

In [100]:
# Almost all classifications went ok
print("Port", rate_portuguese)
print("Eng", rate_english)
print("Span", rate_spanish)

Port 1.0
Eng 1.0
Span 0.97


In [101]:
results_spanish

['spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'english',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'english',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'spanish',
 'sp